In [1]:
!nvidia-smi

Fri Aug 15 04:54:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q "git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/CLIP"
!pip install -q "git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/ml-mobileclip"
!pip install -q "git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/lvis-api"
!pip install -q "git+https://github.com/THU-MIG/yoloe.git"

!pip install -q supervision jupyter_bbox_widget

!wget -q https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_blt.pt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00
  P

In [3]:
from huggingface_hub import hf_hub_download

path = hf_hub_download(repo_id="jameslahm/yoloe", filename="yoloe-v8l-seg.pt", local_dir='.')
path = hf_hub_download(repo_id="jameslahm/yoloe", filename="yoloe-v8l-seg-pf.pt", local_dir='.')

yoloe-v8l-seg.pt:   0%|          | 0.00/107M [00:00<?, ?B/s]

yoloe-v8l-seg-pf.pt:   0%|          | 0.00/103M [00:00<?, ?B/s]

In [4]:
import supervision as sv
from ultralytics import YOLOE
from PIL import Image
import os

folder_path = '/content/drive/MyDrive/VLSP 2025/data/public_test/public_test_images/'
imgs = os.listdir(folder_path)

NAMES = ['sign']
model = YOLOE("yoloe-v8l-seg.pt").cuda()
model.set_classes(NAMES, model.get_text_pe(NAMES))

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Build text model mobileclip:blt


In [5]:
save_folder = '/content/drive/MyDrive/VLSP 2025/enriched_data/public_test/annotated_images/'
crop_folder = '/content/drive/MyDrive/VLSP 2025/enriched_data/public_test/cropped_images/'
local_folder = '/content/drive/MyDrive/VLSP 2025/enriched_data/public_test/local_images/'
os.makedirs(save_folder, exist_ok=True)
os.makedirs(crop_folder, exist_ok=True)
os.makedirs(local_folder, exist_ok=True)

box_annotator = sv.BoxAnnotator(
    color=sv.Color.GREEN,
    thickness=4
)

for img in imgs:
    img_path = os.path.join(folder_path, img)
    image = Image.open(img_path).convert("RGB")
    results = model.predict(image, conf=0.1, verbose=False)

    detections = sv.Detections.from_ultralytics(results[0])

    annotated_image = image.copy()
    annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
    save_path = os.path.join(save_folder, img)
    annotated_image.save(save_path)

    if len(detections.xyxy) > 0:
        x_min = min(d[0] for d in detections.xyxy)
        y_min = min(d[1] for d in detections.xyxy)
        x_max = max(d[2] for d in detections.xyxy)
        y_max = max(d[3] for d in detections.xyxy)

        box_width = x_max - x_min
        box_height = y_max - y_min
        expand_w = (box_width * 1.5 - box_width) / 2
        expand_h = (box_height * 1.5 - box_height) / 2

        x_min = max(0, int(x_min - expand_w))
        y_min = max(0, int(y_min - expand_h))
        x_max = min(image.width, int(x_max + expand_w))
        y_max = min(image.height, int(y_max + expand_h))

        crop_img = image.crop((x_min, y_min, x_max, y_max))
        crop_img.save(os.path.join(crop_folder, img))

        annotated_crop = annotated_image.crop((x_min, y_min, x_max, y_max))
        annotated_crop.save(os.path.join(local_folder, img))
    else:
        image.save(os.path.join(crop_folder, img))
        annotated_image.save(os.path.join(local_folder, img))